In [1]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import username, password
import psycopg2
import sqlalchemy

## Store CSV into DataFrame

In [2]:
#create df from the csv files
path1 = "Resources/film-locations-in-san-francisco-1.csv"
path2 = "Resources/netflix_titles.csv"

netflix_movies = pd.read_csv(path2)
film_locations = pd.read_csv(path1)

In [3]:
#check the df
netflix_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [4]:
#check the df
film_locations.head()

,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,Actor 2,Actor 3
0,180,2011,Epic Roasthouse (399 Embarcadero),NaN,SPI Cinemas,NaN,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand
1,180,2011,Mason & California Streets (Nob Hill),NaN,SPI Cinemas,NaN,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand
2,180,2011,Justin Herman Plaza,NaN,SPI Cinemas,NaN,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand
3,180,2011,200 block Market Street,NaN,SPI Cinemas,NaN,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand
4,180,2011,City Hall,NaN,SPI Cinemas,NaN,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand


## Create new dataframe from selected columns

In [5]:
#new df from specific columns
film_locations_df = film_locations[['Title', 'Release Year', 'Locations']].copy()
film_locations_df.head()

,Title,Release Year,Locations
0,180,2011,Epic Roasthouse (399 Embarcadero)
1,180,2011,Mason & California Streets (Nob Hill)
2,180,2011,Justin Herman Plaza
3,180,2011,200 block Market Street
4,180,2011,City Hall


In [6]:
#new df from specific columns
netflix_df = netflix_movies[['show_id', 'title', 'country', 'release_year']].copy()
netflix_df

,show_id,title,country,release_year
0,81145628,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019
1,80117401,Jandino: Whatever it Takes,United Kingdom,2016
2,70234439,Transformers Prime,United States,2013
3,80058654,Transformers: Robots in Disguise,United States,2016
4,80125979,#realityhigh,United States,2017
...,...,...,...,...
6229,80000063,Red vs. Blue,United States,2015
6230,70286564,Maron,United States,2016
6231,80116008,Little Baby Bum: Nursery Rhyme Friends,NaN,2016
6232,70281022,A Young Doctor's Notebook and Other Stories,United Kingdom,2013


## Connect to database

In [11]:
#assign variables 
username = username
password = password

#connect to local database
rds_connection_string = f"{username}:{password}@localhost:5432/Movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
#check for tables
engine.table_names()

['film_locations', 'netflix']

In [13]:
#convert dataframe to database
film_locations_df.to_sql(name="film_locations", con=engine, if_exists='append',  index=False)

In [14]:
#check the database to confirm
pd.read_sql_query('select * from film_locations', con=engine).head()

,id,Title,Release Year,Locations
0,1,180,2011,Epic Roasthouse (399 Embarcadero)
1,2,180,2011,Mason & California Streets (Nob Hill)
2,3,180,2011,Justin Herman Plaza
3,4,180,2011,200 block Market Street
4,5,180,2011,City Hall


In [15]:
#convert dataframe to database
netflix_df.to_sql(name='netflix', con=engine, if_exists='append', index=False)

In [16]:
#check the database to confirm
pd.read_sql_query('select * from netflix', con=engine).head()

,show_id,title,country,release_year
0,81145628,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019
1,80117401,Jandino: Whatever it Takes,United Kingdom,2016
2,70234439,Transformers Prime,United States,2013
3,80058654,Transformers: Robots in Disguise,United States,2016
4,80125979,#realityhigh,United States,2017
